In [1]:
#import all datasets
import pandas as pd 

admissions = pd.read_csv(r"/workspaces/Predictive-Healthcare-Capacity-Management/Database/admissions.csv")
metrics = pd.read_csv(r"/workspaces/Predictive-Healthcare-Capacity-Management/Database/daily_metrics.csv")
hospitals = pd.read_csv(r"/workspaces/Predictive-Healthcare-Capacity-Management/Database/hospitals.csv")
main_admissions = pd.read_csv(r"/workspaces/Predictive-Healthcare-Capacity-Management/Database/main_admissions.csv")
main_metrics = pd.read_csv(r"/workspaces/Predictive-Healthcare-Capacity-Management/Database/main_daily_metrics.csv")
wards = pd.read_csv(r"/workspaces/Predictive-Healthcare-Capacity-Management/Database/wards.csv")
print(admissions.head())



   ï»¿admission_id        date  hospital_id ward_code arrival_source  \
0                1  2022-10-28            1        ED       referral   
1                2  2022-10-28            1       ICU      ambulance   
2                3  2022-10-28            1        ED           self   
3                4  2022-10-28            1       MED           self   
4                5  2022-10-28            1        ED           self   

  triage_level  wait_minutes  length_of_stay_days      outcome  age sex  \
0          4.0           189                    0   discharged   48   M   
1          4.0           189                    0  transferred   93   F   
2          NaN           189                    0   discharged   52   M   
3          3.0           189                    0   discharged   64   M   
4          4.0           189                    0   discharged   59   M   

   procedure_flag  
0             0.0  
1             0.0  
2             0.0  
3             0.0  
4             0.

In [7]:
admissions

,ï»¿admission_id,date,hospital_id,ward_code,arrival_source,triage_level,wait_minutes,length_of_stay_days,outcome,age,sex,procedure_flag,admission_id
0,1.0,2022-10-28,1,ED,referral,4.0,189,0,discharged,48,M,0.0,NaN
1,2.0,2022-10-28,1,ICU,ambulance,4.0,189,0,transferred,93,F,0.0,NaN
2,3.0,2022-10-28,1,ED,self,NaN,189,0,discharged,52,M,0.0,NaN
3,4.0,2022-10-28,1,MED,self,3.0,189,0,discharged,64,M,0.0,NaN
4,5.0,2022-10-28,1,ED,self,4.0,189,0,discharged,59,M,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
169271,NaN,2024-03-17,5,SURG,self,NaN,104,1,discharged,65,F,1.0,189393.0
169272,NaN,2024-03-17,5,SURG,self,NaN,104,3,discharged,60,F,1.0,189394.0
169273,NaN,2024-03-17,5,SURG,self,NaN,104,5,readmit_30d,81,M,1.0,189395.0
169274,NaN,2024-03-18,5,SURG,ambulance,NaN,105,3,discharged,65,M,1.0,189396.0


Merge Our Data from the two sources

In [2]:
# Concatenate admissions and main_admissions, ignore index to avoid duplicate indices
admissions = pd.concat([admissions, main_admissions], ignore_index=True)

# Concatenate metrics and main_metrics, ignore index to avoid duplicate indices
metrics = pd.concat([metrics, main_metrics], ignore_index=True)


Checking for inconsistencies 

In [21]:
#Checking null values 
list_data = [("admissions",admissions),("metrics",metrics), ("hospitals",hospitals),("wards",wards)]

for name, data in list_data:
    print(name)
    print(f"{round(data.isnull().mean()*100,2)}\n")
    print(data.duplicated().sum())
    print(data.describe())
    print(data.describe(include=("O","category")))
    print("-"*100)

admissions
ï»¿admission_id        99.68
date                    0.00
hospital_id             0.00
ward_code               0.00
arrival_source          0.00
triage_level            0.00
wait_minutes            0.00
length_of_stay_days     0.00
outcome                 0.00
age                     0.00
sex                     0.00
procedure_flag          0.00
admission_id            0.32
dtype: float64

0
       ï»¿admission_id                           date    hospital_id  \
count        545.00000                         169276  169276.000000   
mean         273.00000  2024-04-14 13:36:57.131784960       2.861197   
min            1.00000            2022-10-28 00:00:00       1.000000   
25%          137.00000            2023-07-12 00:00:00       2.000000   
50%          273.00000            2024-04-07 00:00:00       3.000000   
75%          409.00000            2025-01-16 00:00:00       4.000000   
max          545.00000            2025-10-26 00:00:00       5.000000   
std          157.4

Calculating Empirical outliers

-mean #formular depends on the thresholds you want to set



-standard deviation # for less stricter threshold one can use one sd but normally can be two/three sd


#calculating for the lower threshold
lower = mean -(3*standard deviation) # anything below the lower boudary is an outlier 
lower = mean -(3*standard deviation) #anything above the upper boundary is an outlier 

In [4]:
for name, data in list_data:
    print(name)
    for col in data.select_dtypes(include='number').columns.tolist():
        mean = data[col].mean()
        std = data[col].std()

        lower = mean - (3 * std)
        upper = mean + (3 * std)

        outliers = data[(data[col] < lower) | (data[col] > upper)]
        print(f"Outliers in column '{col}':")
        print(outliers)

        print(col)
        print(len(outliers))
        print('-'*50)
        print('_'*50)


admissions
Outliers in column 'ï»¿admission_id':
Empty DataFrame
Columns: [ï»¿admission_id, date, hospital_id, ward_code, arrival_source, triage_level, wait_minutes, length_of_stay_days, outcome, age, sex, procedure_flag, admission_id]
Index: []
ï»¿admission_id
0
--------------------------------------------------
__________________________________________________
Outliers in column 'hospital_id':
Empty DataFrame
Columns: [ï»¿admission_id, date, hospital_id, ward_code, arrival_source, triage_level, wait_minutes, length_of_stay_days, outcome, age, sex, procedure_flag, admission_id]
Index: []
hospital_id
0
--------------------------------------------------
__________________________________________________
Outliers in column 'wait_minutes':
        ï»¿admission_id        date  hospital_id ward_code arrival_source  \
192               193.0  2022-11-13            1        ED           self   
193               194.0  2022-11-13            1        ED      ambulance   
194               195

In [5]:
# List of columns to drop
cols_to_drop = ['admissions', 'date']

for name, data in list_data:
    print(f"Processing {name} dataframe:")
    # Drop columns only if they exist in the dataframe
    data = data.drop(columns=[col for col in cols_to_drop if col in data.columns], errors='ignore')
    
    # Print the dataframe after dropping the columns
    print(f"Data after dropping columns: {cols_to_drop}")
    print(data.head())  # Preview the updated data
    print("-" * 50)


Processing admissions dataframe:
Data after dropping columns: ['admissions', 'date']
   ï»¿admission_id  hospital_id ward_code arrival_source triage_level  \
0              1.0            1        ED       referral          4.0   
1              2.0            1       ICU      ambulance          4.0   
2              3.0            1        ED           self          NaN   
3              4.0            1       MED           self          3.0   
4              5.0            1        ED           self          4.0   

   wait_minutes  length_of_stay_days      outcome  age sex  procedure_flag  \
0           189                    0   discharged   48   M             0.0   
1           189                    0  transferred   93   F             0.0   
2           189                    0   discharged   52   M             0.0   
3           189                    0   discharged   64   M             0.0   
4           189                    0   discharged   59   M             0.0   

   admi

###for study purposes [fill null values]


-forward fill



-backward fill


-grouped mode imputation


-KNN imputer



-interpolation 

###Deal with the Outliers (What, why and when)

-Winsorization



-Scaling/Log transformation 


would be great to get a timer function 

In [6]:
 admissions['triage_level'].value_counts()

triage_level
3.0    21068
4.0    15812
2.0     7897
5.0     5217
1.0     2566
3.0      214
4.0      155
2.0       76
5.0       69
1.0       28
6.5        1
abc        1
Name: count, dtype: int64

In [9]:
admissions['ward_code'].nunique()

5

In [14]:
admissions['arrival_source'].nunique()

4

In [12]:
values_to_replace = {
    "6.5": "Null",
    "abc": "Null"
}

admissions['triage_level'].replace(values_to_replace, inplace=True)


In [15]:
admissions['triage_level'].mode() #wont really reduce biasessness 

0    3.0
Name: triage_level, dtype: object

we are ging to look for columns if they exist 

In [20]:
cols_to_fill = ['triage_level', 'date']

for name, data in list_data:
    for col in cols_to_fill:
        if 'triage_level' in data.columns:
            data['triage_level'] = data.groupby('ward_code')['triage_level'].transform(lambda x: x.fillna(x.mode()[0]))
            
            if 'date' in data.columns:
                data['date'] = pd.to_datetime(data['date'], errors='coerce')
                data['date'] = data['date'].fillna(method='ffill')
                data['date'] = data['date'].fillna(method='bfill')




/tmp/ipykernel_12798/3295746549.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['triage_level'] = data.groupby('ward_code')['triage_level'].transform(lambda x: x.fillna(x.mode()[0]))
/tmp/ipykernel_12798/3295746549.py:10: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['date'] = data['date'].fillna(method='ffill')
/tmp/ipykernel_12798/3295746549.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['date'] = data['date'].fillna(method='bfill')
/tmp/ipykernel_12798/3295746549.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will cha